In [8]:
import numpy as np
import tensorflow as tf

from keras.utils import load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout
from keras import Model

In [3]:
def init_vgg16():

    # Download the model with weights pre-trained using ImageNet database
    vgg16 = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(224,224,3))
    
    
    # Freeze layers for training
    for layer in vgg16.layers:
        layer.trainable = False

    # Create a new 'top' of the model of fully-connected layers
    top_model = vgg16.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu', name="top_dense_1")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(512, activation='relu', name="top_dense_2")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(256, activation='relu', name="top_dense_3")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(128, activation='relu', name="top_dense_4")(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(1, activation='softmax', name="output" )(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=vgg16.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [4]:
model = init_vgg16()
model.summary()

2023-04-17 11:31:46.368423: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-17 11:31:46.368916: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
img_path = 'elephant.jpg'
img = load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

FileNotFoundError: [Errno 2] No such file or directory: 'elephant.jpg'

In [20]:
features[0].shape

NameError: name 'features' is not defined

In [16]:
tf.keras.applications.vgg16.decode_predictions(
    features, top=5
)

35363/35363 [==============================] - 0s 1us/step


[[('n02504458', 'African_elephant', 0.89311653),
  ('n01871265', 'tusker', 0.063906215),
  ('n02504013', 'Indian_elephant', 0.04295029),
  ('n01704323', 'triceratops', 1.2360962e-05),
  ('n02437312', 'Arabian_camel', 5.3780045e-06)]]